# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [11]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [ ]:
이 코드는 OpenAI API를 쓰기 위해 환경설정과 키 점검을 하는 초기화 부분이에요.
단계별로 풀면 이렇게 됩니다.

환경변수 불러오기

python
코드 복사
load_dotenv(override=True)
.env 파일에 저장된 환경변수를 파이썬으로 불러옵니다.

override=True는 기존 환경변수가 있어도 덮어쓴다는 의미예요.

API 키 가져오기

python
코드 복사
api_key = os.getenv('OPENAI_API_KEY')
불러온 환경변수 중에서 OPENAI_API_KEY 값을 읽어옵니다.

이 값이 OpenAI API를 호출할 때 필요한 비밀번호 같은 거예요.

API 키 유효성 점검

python
코드 복사
if api_key and api_key.startswith('sk-proj-') and len(api_key) > 10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
API 키가 존재하는지 확인 (api_key)

키가 'sk-proj-'로 시작하는지 확인 (OpenAI 프로젝트 키 형식)

길이가 10자보다 긴지 확인 (너무 짧으면 잘못된 키일 가능성)

조건이 맞으면 "API key looks good so far" 출력,
아니면 "There might be a problem..." 경고 메시지 출력.

모델 이름 설정

python
코드 복사
MODEL = 'gpt-4o-mini'
사용할 OpenAI 모델을 지정합니다.

여기선 'gpt-4o-mini'를 선택했네요.

OpenAI 객체 생성

python
코드 복사
openai = OpenAI()
OpenAI API 호출을 도와주는 객체를 만듭니다.

이후 openai.chat.completions.create(...) 같은 메서드로 요청을 보낼 수 있어요.

💡 쉽게 비유
이건 마치

.env에서 비밀번호 찾아오기

그 비밀번호가 맞는지 대충 검사하기

사용할 모델 정하기

OpenAI에 연결 준비 완료!
… 이런 준비 단계입니다.



In [2]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [ ]:
이 코드는 웹페이지를 가져와서 제목, 본문 텍스트, 링크를 추출하는 도우미 클래스예요.
단계별로 풀어보면 이렇게 됩니다.

1. 웹 요청 헤더 설정
python
코드 복사
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) ..."
}
일부 웹사이트는 그냥 requests로 접속하면 로봇으로 인식해서 차단합니다.

그래서 브라우저(크롬)에서 온 것처럼 속이는 User-Agent를 추가해줍니다.

2. Website 클래스
python
코드 복사
class Website:
    """웹페이지의 제목, 본문, 링크를 저장하는 클래스"""
하나의 웹페이지를 표현하는 객체.

URL을 넣으면 해당 페이지를 가져와서 필요한 정보를 자동으로 뽑아옵니다.

3. 초기화 메서드 (__init__)
python
코드 복사
def __init__(self, url):
    self.url = url
사용자가 Website("https://example.com")처럼 URL을 넣으면 실행됩니다.

주요 기능:

웹페이지 요청

python
코드 복사
response = requests.get(url, headers=headers)
self.body = response.content
→ URL의 HTML 내용을 가져옵니다.

HTML 파싱

python
코드 복사
soup = BeautifulSoup(self.body, 'html.parser')
→ BeautifulSoup으로 HTML을 구조적으로 분석.

제목 추출

python
코드 복사
self.title = soup.title.string if soup.title else "No title found"
→ <title> 태그가 있으면 제목 저장, 없으면 "No title found".

본문 텍스트 정리

python
코드 복사
for irrelevant in soup.body(["script", "style", "img", "input"]):
    irrelevant.decompose()
self.text = soup.body.get_text(separator="\n", strip=True)
→ script, style, img, input 같은 필요 없는 태그 삭제 후,
본문 텍스트만 깔끔하게 추출.

링크 수집

python
코드 복사
links = [link.get('href') for link in soup.find_all('a')]
self.links = [link for link in links if link]
→ <a> 태그의 href 속성 값을 리스트로 저장 (빈 값 제외).

4. 콘텐츠 반환 메서드
python
코드 복사
def get_contents(self):
    return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"
저장된 제목과 본문 텍스트를 문자열 형태로 돌려줍니다.

💡 쉽게 비유
이건 마치 웹페이지 스캐너예요.

Website(url) → 그 페이지의 HTML을 받아와서

제목, 텍스트, 링크를 자동으로 뽑고 저장

필요하면 get_contents()로 깔끔하게 내용 보기

In [3]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [ ]:
첫 번째 단계: GPT-4o-mini로 어떤 링크가 관련 있는지 판단하기
gpt-4o-mini를 호출하여 웹페이지의 링크를 읽고, 구조화된 JSON으로 응답하게 합니다.
이 과정에서 어떤 링크가 관련 있는지를 판단하고, "/about" 같은 상대 경로 링크를 "https://company.com/about"처럼 절대 경로로 변환해야 합니다.
우리는 One-shot 프롬프트 기법을 사용할 것이며, 프롬프트에 모델이 어떻게 응답해야 하는지 예시를 함께 제공합니다.

이 작업은 LLM의 훌륭한 활용 사례입니다. 왜냐하면 미묘한 맥락 이해가 필요하기 때문입니다. LLM 없이 단순히 코드를 작성해 웹페이지를 파싱하고 분석하려고 하면, 매우 어려운 작업이 될 것입니다.

참고: 더 고급 기법으로 Structured Outputs가 있습니다. 이 방법은 모델이 특정 스펙에 맞춰 응답하도록 요구합니다. 이 기법은 8주차 자율형 Agentic AI 프로젝트에서 다룰 예정입니다.

In [4]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
} 
"""
###원샷 프롬프트 

In [30]:
##멀티샷 프롬프트
link_system_prompt = """You are provided with a list of links found on a webpage.
Choose links relevant for a company brochure (About, Company, Careers/Jobs, Press).
Exclude Terms, Privacy, email links, login, and ads. Always return absolute https URLs.
Respond ONLY in JSON like:
{ "links": [ { "type": "<about|careers|...>", "url": "https://..." } ] }

# Example 1
Input (base: https://companyA.com):
/about
/contact
/privacy
/terms
mailto:hello@companyA.com
/careers

Output:
{
  "links": [
    {"type": "about page", "url": "https://companyA.com/about"},
    {"type": "careers page", "url": "https://companyA.com/careers"}
  ]
}

# Example 2
Input (base: https://www.startup.io):
https://www.startup.io/company
/login
/products
/newsroom
/privacy-policy
/jobs

Output:
{
  "links": [
    {"type": "company page", "url": "https://www.startup.io/company"},
    {"type": "careers page", "url": "https://www.startup.io/jobs"},
    {"type": "press/newsroom page", "url": "https://www.startup.io/newsroom"}
  ]
}
"""


In [5]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [6]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [ ]:
이 코드는 웹사이트의 링크 목록을 GPT 같은 모델에 보낼 프롬프트(질문문)를 만드는 함수예요.
단계별로 풀면 이렇게 됩니다.

1. 함수 정의
python
코드 복사
def get_links_user_prompt(website):
website는 앞서 만든 Website 클래스 같은 객체일 가능성이 큽니다.

즉, website.url → 해당 웹사이트 주소

website.links → 그 사이트에서 수집한 링크 목록

2. 기본 프롬프트 시작
python
코드 복사
user_prompt = f"Here is the list of links on the website of {website.url} - "
프롬프트 첫 부분에 "이 사이트({website.url})의 링크 목록입니다"라는 문장을 작성.

3. 요청 내용 추가
python
코드 복사
user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
모델에게 지시:

회사 브로셔에 쓸만한 관련 있는 링크만 고르기

전체 HTTPS URL로 JSON 형식으로 응답하기

이용 약관, 개인정보 처리방침, 이메일 링크는 제외하기

4. 링크 목록 안내 문구
python
코드 복사
user_prompt += "Links (some might be relative links):\n"
다음에 나올 링크들이 일부는 /about처럼 상대 경로일 수 있다는 걸 알려줌.

5. 실제 링크들 추가
python
코드 복사
user_prompt += "\n".join(website.links)
website.links 리스트의 모든 링크를 줄바꿈으로 이어붙임.

6. 완성된 프롬프트 반환
python
코드 복사
return user_prompt
완성된 GPT 요청용 문자열을 돌려줌.

💡 쉽게 말하면
이 함수는

“이 사이트 링크 목록을 줄 테니, 브로셔에 필요한 링크만 골라서 JSON으로 줘”
라는 요청문을 자동으로 만들어주는 프롬프트 생성기입니다.

원하면 제가 이 함수를 실행했을 때 실제로 만들어지는 프롬프트 예시를 하나 보여드릴 수 있습니다.
그렇게 해드릴까요?





나의 말:
응 예시도 보여줘


ChatGPT의 말:
좋아요 👍
예를 들어, Website 객체가 이렇게 생겼다고 가정해볼게요.

python
코드 복사
website = Website("https://company.com")
website.links = [
    "/about",
    "/products",
    "/contact",
    "/privacy-policy",
    "https://company.com/blog",
    "mailto:info@company.com"
]
이 상태에서 함수를 실행하면:

python
코드 복사
print(get_links_user_prompt(website))
실제 결과 예시
swift
코드 복사
Here is the list of links on the website of https://company.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/about
/products
/contact
/privacy-policy
https://company.com/blog
mailto:info@company.com
📌 이 프롬프트를 GPT에 보내면 모델은 이런 식으로 응답할 수 있어요:

json
코드 복사
[
  "https://company.com/about",
  "https://company.com/products",
  "https://company.com/contact",
  "https://company.com/blog"
]
→ privacy-policy나 mailto: 같은 건 제외하고, 상대 경로는 절대 경로로 변환해서 줍니다.

In [7]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [8]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}    ###RESPONSE 포맷 설정 가능, 프롬프트에도 명시해야 동작
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [ ]:
좋아요! 이 한 줄이 하는 일을 쪼개서 아주 쉽게 설명해볼게요.

핵심 요약
response 안에는 모델의 여러 후보 답변(choices) 이 들어 있습니다.

choices[0]는 그중 첫 번째 답변이고, .message.content가 그 답변의 순수 텍스트 내용입니다.

그래서 result는 모델이 출력한 텍스트 문자열이 됩니다.

쉬운 비유
모델이 답안을 여러 장 제출할 수도 있어요(시험지 여러 버전). choices는 그 답안 묶음이고, choices[0]는 1번 답안지, 그 안의 message.content는 실제 적힌 글이에요. 우리는 “첫 번째 답안지의 글”을 꺼내 result에 담는 셈이죠.

구조로 보기 (응답 객체의 전형적인 형태)
scss
코드 복사
response
└─ choices  ← 리스트(여러 개일 수 있음)
   ├─ [0]
   │   ├─ index
   │   ├─ message
   │   │   ├─ role ("assistant")
   │   │   └─ content  ← 모델이 만든 텍스트
   │   └─ finish_reason
   └─ [1], [2], ...
코드 한 줄 해석
python
코드 복사
result = response.choices[0].message.content
response.choices : 모델 답변 후보들의 리스트

[0] : 첫 번째 후보 선택

.message : 그 후보의 메시지 객체

.content : 그 메시지의 텍스트(스트링)

실전 팁 & 주의사항
여러 후보를 요청한 경우
n=3 같은 옵션으로 여러 답변을 받았다면, [1], [2]에서도 message.content를 꺼낼 수 있어요.

함수/툴 호출 응답
일부 호출(예: tool/function call)에서는 content가 비어 있고 tool_calls만 있을 수 있어요. 그땐 아래처럼 안전하게 처리하세요.

python
코드 복사
msg = response.choices[0].message
text = msg.content or ""
if getattr(msg, "tool_calls", None):
    # 여기서 툴 호출 정보를 처리
    pass
스트리밍 모드(stream=True)
스트리밍이면 response가 이벤트 스트림 형태라 이 한 줄로는 못 꺼냅니다. 청크를 모아 이어붙여야 해요.

다른 API와 혼동 주의
예전 Completions API는 choices[0].text였고, Chat Completions은 choices[0].message.content입니다. 헷갈리면 문서를 확인하세요.

원하면 지금 쓰고 있는 호출 코드에 맞춘 견고한 추출 함수(단일/다중 후보, 툴콜/스트리밍 대비)도 만들어 드릴게요.







ChatGPT에게 묻기


In [9]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/openai/gpt-oss-120b',
 '/openai/gpt-oss-20b',
 '/Qwen/Qwen-Image',
 '/tencent/Hunyuan-1.8B-Instruct',
 '/rednote-hilab/dots.ocr',
 '/models',
 '/spaces/Qwen/Qwen-Image',
 '/spaces/enzostvs/deepsite',
 '/spaces/black-forest-labs/FLUX.1-Krea-dev',
 '/spaces/Wan-AI/Wan-2.2-5B',
 '/spaces/alexnasa/OmniAvatar',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/jxm/gpt-oss20b-samples',
 '/datasets/HuggingFaceH4/Multilingual-Thinking',
 '/datasets/openai/BrowseCompLongContext',
 '/datasets/nvidia/Nemotron-Post-Training-Dataset-v1',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/

In [12]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()  #해당 URL의 본문
    links = get_links(url)  ## 모델 써서 관련있는 링크들 불러와
    print("Found links:", links) # 그거 프린트
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()  #각 링크에 대한 컨텐츠 리절트에 추가
    return result

In [ ]:
좋아요 👍
이 함수는 웹사이트 메인 페이지와, 그 안에서 뽑아낸 관련 링크들의 페이지 내용을 한 번에 모아주는 함수예요.
단계별로 설명해 드릴게요.

1. 함수 정의
python
코드 복사
def get_all_details(url):
url : 조사할 웹사이트 주소 (예: "https://company.com").

2. 결과 문자열 초기화
python
코드 복사
result = "Landing page:\n"
result라는 문자열을 만들고, 맨 앞에 "Landing page:\n"를 넣어둡니다.

나중에 여기에 각 페이지의 내용을 계속 덧붙일 거예요.

3. 메인 페이지 내용 가져오기
python
코드 복사
result += Website(url).get_contents()
Website(url) → 앞에서 만든 Website 클래스로 해당 URL 페이지를 불러옴.

.get_contents() → 그 페이지의 제목과 본문 텍스트를 문자열로 가져옴.

이걸 result에 이어붙임.

4. 관련 링크 가져오기
python
코드 복사
links = get_links(url)
get_links(url) : GPT 같은 모델을 써서 메인 페이지에서 회사 브로셔에 쓸만한 관련 링크들만 뽑아주는 함수.

반환값 예시:

python
코드 복사
{
    "links": [
        {"type": "About", "url": "https://company.com/about"},
        {"type": "Products", "url": "https://company.com/products"}
    ]
}
5. 찾은 링크 출력
python
코드 복사
print("Found links:", links)
콘솔에 어떤 링크를 찾았는지 확인용으로 출력.

6. 각 링크 페이지 내용 추가
python
코드 복사
for link in links["links"]:
    result += f"\n\n{link['type']}\n"
    result += Website(link["url"]).get_contents()
links["links"] 안에 있는 링크들을 하나씩 꺼내서:

link['type'] : 페이지 유형(About, Products 등) 제목을 result에 추가

Website(link["url"]).get_contents() : 그 페이지의 제목과 본문 텍스트를 가져와서 result에 이어붙임.

7. 최종 결과 반환
python
코드 복사
return result
메인 페이지 + 관련 링크 페이지들의 내용을 한 덩어리로 합친 문자열을 돌려줍니다.

💡 쉽게 비유
이건 **“홈페이지랑, 중요한 다른 페이지들까지 한 번에 스크랩해서 한 문서로 모아주는 수집기”**예요.

1단계 : 메인 페이지 내용 수집

2단계 : AI가 중요한 링크 추천

3단계 : 그 링크 페이지들 내용 수집

4단계 : 전부 합쳐서 하나의 결과물로 반환

In [ ]:
그럼 get_all_details()가 실행됐을 때 나올 법한 예시를 만들어드릴게요.

가정
URL: "https://company.com"

get_links(url)이 반환하는 값:

python
코드 복사
{
    "links": [
        {"type": "About", "url": "https://company.com/about"},
        {"type": "Products", "url": "https://company.com/products"}
    ]
}
각 페이지에는 간단한 제목과 본문이 있다고 가정.

실행 코드
python
코드 복사
output = get_all_details("https://company.com")
print(output)
결과 예시
yaml
코드 복사
Landing page:
Webpage Title:
Company - Welcome
Webpage Contents:
Welcome to our company website.
We offer a range of products and services to suit your needs.

Found links: {
    "links": [
        {"type": "About", "url": "https://company.com/about"},
        {"type": "Products", "url": "https://company.com/products"}
    ]
}

About
Webpage Title:
About Us - Company
Webpage Contents:
Our company was founded in 2010 with the mission to deliver high-quality products worldwide.
We value innovation, customer satisfaction, and sustainability.

Products
Webpage Title:
Our Products - Company
Webpage Contents:
We offer electronics, home appliances, and software solutions.
Visit our online store to learn more about each category.
📌 흐름 정리

"Landing page:" → 메인 페이지 제목과 본문

"Found links: ..." → AI가 고른 관련 링크 출력

각 링크의 type과 그 페이지의 제목·본문을 순서대로 추가

In [14]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'support page', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
openai/gpt-oss-120b
Updated
3 days ago
•
429k
•
3.16k
openai/gpt-oss-20b
Updated
3 days ago
•
2M
•
2.73k
Qwen/Qwen-Image
Updated
5 days ago
•
62.8k
•
1.42k
tencent/Hunyuan-1.8B

In [28]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. You are recruiting Korean workers, so \
you have to make brochure which are written by Korean. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# 한국어 브로셔로 변경한 프롬프트

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)  # 메인페이지+ 관련 페이지까지 다 1개로 합친 거 
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [17]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'Github', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'}]}


"You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nopenai/gpt-oss-120b\nUpdated\n3 days ago\n•\n429k\n•\n3.16k\nopenai/gpt-oss-20b\nUpdated\n3 days ago\n•\n2M\n•\n2.73k\nQwen/Qwen-Image\nUpdated\n5 days ago\n•\n62.8k\n•\n1.43k\ntencent/Hunyuan-1.8B-Instruct\nUpdated\n5 days ago\n•\n3.1k\n•\n573\nrednote-hilab/dots.ocr\nUpdated\n4 days ago\n•\n16.2k\n•\n563\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\n415\n415\nQwen Image\n🖼\nGenerate ima

In [18]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [19]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


```markdown
# Hugging Face: The AI Community Building the Future

Welcome to Hugging Face, a pioneering platform that empowers the machine learning community through collaboration, innovation, and shared resources. Our mission is to accelerate the development of AI technologies and democratize access to machine learning tools for everyone.

## What We Offer

- **Models**: Access over 1 million machine learning models, including cutting-edge architectures like GPT, diffusion models, and more.
  
- **Datasets**: Explore a vast library of over 250,000 datasets tailored for various machine learning tasks.

- **Spaces**: Utilize our user-friendly interface to create and share applications, allowing for seamless collaboration and innovation.

- **Enterprise Solutions**: We provide tailored enterprise-grade services, ensuring security, access control, and dedicated support for organizations looking to harness the power of AI.

## Our Community

Hugging Face is not just a platform, but a vibrant community of over 50,000 organizations and many active users. Major companies like Google, Amazon, Microsoft, and Grammarly utilize our tools and models, demonstrating our commitment to serving both individual developers and large enterprises.

### Collaboration at Our Core

We believe in the power of community and open-source. Our projects like Transformer models, Diffusers, and Tokenizers are designed for collaborative building and sharing, creating a robust ecosystem for machine learning developers.

## Company Culture

At Hugging Face, we foster a culture of inclusivity, innovation, and continuous improvement. We encourage our team members to explore new ideas, collaborate across functions, and contribute to an open-source community that thrives on shared knowledge.

- **Team Size**: Currently featuring over 211 dedicated professionals passionate about AI and machine learning.
  
- **Work Environment**: We promote flexibility, creativity, and a consultancy-like atmosphere where every team member's input is valued.

## Careers at Hugging Face

Join a team that is shaping the future of AI! We are always on the lookout for talented individuals who are excited about contributing to impactful projects. 

### Positions Available:
- Machine Learning Engineers
- Data Scientists
- Software Developers
- Community Managers
- Product Designers

Explore opportunities on our [Careers Page](https://huggingface.co/jobs) and find out how you can become part of the Hugging Face family.

## Getting Started

Whether you’re an AI enthusiast, developer, or someone looking for robust machine learning solutions, Hugging Face is the place to be. 

- **Explore AI Apps**: Dive deep into our extensive application ecosystem.
- **Sign Up**: Create an account to start building or collaborating on models and datasets.

**Hugging Face** - Building the AI community of the future, together.

🔗 [Visit Our Website](https://huggingface.co)  
🔗 [Join Us on GitHub](https://github.com/huggingface)  
🔗 [Follow Us on Twitter](https://twitter.com/huggingface)  
🔗 [Connect with Us on LinkedIn](https://linkedin.com/company/huggingface/)  
```


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
이 함수 stream_brochure()는 OpenAI API의 스트리밍 모드를 사용해서, 회사 브로셔 내용을 실시간으로 화면에 출력하는 기능이에요.
단계별로 풀어보면 이렇게 됩니다.

1. 함수 정의
python
코드 복사
def stream_brochure(company_name, url):
매개변수

company_name: 회사 이름 (예: "OpenAI")

url: 해당 회사 웹사이트 주소 (예: "https://openai.com")

2. GPT 호출 준비 (스트리밍 모드)
python
코드 복사
stream = openai.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
      ],
    stream=True
)
model=MODEL : 사용할 모델 이름(예: "gpt-4o-mini").

messages : 챗봇 대화 맥락

첫 번째 메시지 (system) → 전체적인 역할·규칙을 설명하는 시스템 프롬프트

두 번째 메시지 (user) → get_brochure_user_prompt()로 만든 사용자 요청(브로셔 작성 지시문)

stream=True : 모델이 한 번에 전부 응답하는 게 아니라 조각(chunk) 단위로 실시간 전송하도록 함.

3. 초기 화면 출력 준비
python
코드 복사
response = ""
display_handle = display(Markdown(""), display_id=True)
response : 지금까지 받은 텍스트를 누적할 변수.

display_handle : 화면에 마크다운 형태로 표시할 수 있는 영역을 생성(예: Jupyter Notebook 환경).

4. 스트리밍 응답 처리
python
코드 복사
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)
for chunk in stream → GPT가 보내주는 **조각(chunk)**을 하나씩 받음.

chunk.choices[0].delta.content → 이 조각의 새로 생성된 텍스트 부분
(or ''는 None 방지)

response += ... → 전체 응답에 새 텍스트 누적.

.replace("```","").replace("markdown", "") → 마크다운 코드 블록 표식 제거.

update_display(...) → 화면을 실시간으로 갱신해서, 지금까지 받은 내용 보여줌.

💡 쉽게 말하면

stream=True 덕분에 GPT가 “답변 다 썼습니다” 하고 기다리는 게 아니라,
쓰자마자 바로바로 화면에 반영할 수 있는 거예요.

마치 실시간 자막처럼 브로셔 내용을 타이핑하듯 표시하는 거죠.

📌 흐름 요약

GPT 모델에 “이 회사에 대한 브로셔 써줘” 요청 보냄 (스트리밍 모드)

모델이 한 문장씩 보내주면 즉시 누적

화면을 계속 갱신해서 실시간으로 내용 표시



In [31]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'press page', 'url': 'https://huggingface.co/blog'}]}



# Hugging Face 소개 브로셔

## 회사 개요
Hugging Face는 인공지능(AI) 커뮤니티가 미래를 함께 구축하는 플랫폼입니다. 우리는 기계 학습(ML) 커뮤니티가 모델, 데이터 세트 및 애플리케이션에 협업할 수 있도록 도와줍니다. Hugging Face는 전 세계 5만 개 이상의 조직에서 사용되고 있으며, 기업급 보안과 전용 지원을 통해 ML 솔루션을 제공하고 있습니다.

## 회사 문화
Hugging Face는 협력과 혁신을 중시하는 문화가 특징입니다. 우리는 오픈 소스 정신을 통해 커뮤니티와 함께 발전하며, 누구나 쉽게 기여할 수 있는 환경을 만듭니다. 다양성과 포용성을 강조하여, 소외되지 않는 AI 생태계를 조성하는 데 힘쓰고 있습니다.

## 고객
우리는 **Meta**, **Amazon**, **Google**, **Microsoft** 등 여러 글로벌 기업과 협력하고 있습니다. 또한, 다양한 비영리 기관 및 스타트업과 함께 일하며, 그들의 AI 프로젝트를 지원합니다.

## 경력 기회
Hugging Face에서는 다음과 같은 다양한 직무를 제공합니다:
- 기계 학습 엔지니어
- 데이터 과학자
- 소프트웨어 개발자
- 고객 지원 전문인

우리는 모든 기술 수준과 경험을 가진 지원자를 환영합니다! 함께 AI의 미래를 만들어갈 인재를 찾고 있습니다.

## 우리가 제공하는 것
- **모델 및 데이터 세트의 광범위한 라이브러리**: 100만 개 이상의 모델과 25만 개 이상의 데이터 세트에 접근할 수 있습니다.
- **ML 포트폴리오 구축**: 여러분의 작업을 세계와 공유하세요.
- **전문 지원 및 독점 도구**: 팀과 기업을 위한 고급 플랫폼을 제공하며, 안전하고 효율적인 AI 개발을 지원합니다.

저희와 함께 AI의 미래를 창조해 보세요! 더 자세한 사항은 [Hugging Face 웹사이트](https://huggingface.co) 를 방문해 주시기 바랍니다.

## 연락처
Hugging Face에 대한 더 많은 정보와 경력에 대한 문의는 아래의 링크를 통해 가능합니다.
- [채용 정보](https://huggingface.co/jobs)



In [29]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Hugging Face 브로셔

## AI 커뮤니티의 미래를 함께합니다.

Hugging Face는 기계 학습 커뮤니티를 위한 플랫폼으로, 모델, 데이터셋, 응용 프로그램을 협력하여 개발하는 공간입니다. 우리는 누구나 AI 기술에 기여하고 공유할 수 있는 열린 환경을 제공합니다.

## 우리의 자랑스러운 제공안

- **모델**: 100만 개 이상의 최신 AI 모델에 접근하고 탐색하세요.
- **데이터셋**: 25만 개 이상의 공공 데이터셋으로 기계 학습 작업을 지원합니다.
- **Spaces**: AI 애플리케이션을 생성하고 공유하는데 최적화된 공간입니다.

## 고객과의 협력

Hugging Face는 Amazon, Google, Microsoft 등 50,000개 이상의 조직과 협력하고 있으며, 비영리 기업들도 우리 플랫폼을 통해 혁신을 이루고 있습니다. 우리의 성공적인 고객들은 다양한 산업에 걸쳐 있으며, 그들의 필요에 맞춘 AI 솔루션을 제공합니다.

## 포괄적인 회사 문화

Hugging Face에서의 근무는 굉장히 협력적이고 혁신적입니다. 우리는 여러 백그라운드를 가진 인재들이 자유롭게 의견을 교환하고, 아이디어를 실현할 수 있는 환경을 조성합니다. 각 팀원은 자신의 의견이 존중받고, 혁신적인 프로젝트에 참여할 기회를 가집니다.

## 경력 및 채용 정보

우리는 지속적인 성장을 위해 새로운 인재를 찾고 있습니다. 다양한 직무가 있으며, 모두가 AI의 미래를 함께 만들어가는 여정에 동참할 수 있습니다. 특히 한국어를 구사하는 인재를 우대하며, 다국적 팀에서 경험을 쌓을 수 있는 기회를 제공합니다.

## 함께 미래를 만들어 갑니다

Hugging Face는 현대 AI 기술의 최전선에서 활동하고 있으며, 여러분의 참여를 통해 더 나은 AI 생태계를 만들고자 합니다. 관심이 있으신 분들은 [신청하기](https://huggingface.co/join) 버튼을 통해 지원해 주시기 바랍니다!

---

**연락처 정보**  
- 웹사이트: [Hugging Face](https://huggingface.co)  
- 이메일: info@huggingface.co  
- 소셜 미디어: [Twitter](https://twitter.com/huggingface) | [LinkedIn](https://linkedin.com/company/huggingface)



<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>